In [1]:
import tkinter as tk
import time
from tkinter import messagebox

In [2]:
class Tablero():
    
    #caracteres para representar las fichas de cada jugador.
    J1="o"
    J2="Δ"
    
    def __init__(self, tamaño=None):
        #tamaño del tablero. En el reversi es 8x8
        if tamaño==None:
            tamaño=8
            
        self.tamaño=tamaño
        
        self.fin=False
        self.info=""
        self.jugada_viable=True
        self.turno_saltado=False
        
        #Colocar fichas en el tablero en posición inicial, y jugador que empieza
        self.estado=[[0 for _ in range(tamaño)] for _ in range(tamaño)]
        self.estado[tamaño//2-1][tamaño//2-1]=Tablero.J2
        self.estado[tamaño//2][tamaño//2-1]=Tablero.J1
        self.estado[tamaño//2-1][tamaño//2]=Tablero.J1
        self.estado[tamaño//2][tamaño//2]=Tablero.J2
        
        self.estado_anterior=[self.estado[i].copy() for i in range(len(self.estado))]
        self.turno=Tablero.J1
    
    
    
    def _turno_opuesto(self, turno=None):
        if turno==None:
            turno=self.turno
        
        if turno==Tablero.J1:
            return(Tablero.J2)
        else:
            return(Tablero.J1)
            
    def _cambioturno(self):
        self.turno=self._turno_opuesto()
    
    
    def estavacío(self,i,j):
        return(self.estado[i][j]==0)

    
    #busca en las casillas adyacentes a la casilla dada fichas del tipo especificado
    def _buscar_adyacentes(self,i,j,t=None):
        #if t==None:
        #   t=self.turno
               
        lis=[]
        for k in range(-1,2): #posicion inicial, una menos y una más
            for l in range(-1,2):
                x=i+k
                y=j+l

                if range(self.tamaño).__contains__(x) and range(self.tamaño).__contains__(y):
                    if self.estado[x][y]==t:
                        lis+=[(k,l)]
        
        if lis.__contains__((0,0)):
            lis.remove((0,0))
        
        return(lis)
    
    def _efecto_dir(self,i,j,h,k):
        #(i,j) punto inicial
        #(h,k) dirección de la acción
        
        t=self.turno        
        self.estado[i][j]=t
        
        x=h+i
        y=k+j

        while range(self.tamaño).__contains__(x) and range(self.tamaño).__contains__(y):
            #desplazaremos sin salirnos del tablero
            
            if self.estado[x][y]==t or self.estado[x][y]==0:
                return

            self.estado[x][y]= self.turno
            x+=h
            y+=k   
    
    # Una vez determinadas direcciones en que se podría propagar la jugada atendiendo a la primera ficha...
    # ... en la direccion escogida, comprobar si cumple el resto de condiciones para poder girar fichas
    def _direccion_viable(self,i,j,h,k):
        
        t=self.turno
        t2=self._turno_opuesto()
        viable=False

        #posición inicial
        x,y=i+h,j+k
        
        #buscando ficha del mismo tipo en la dirección especificada, salvo que haya huecos
        while (x in range(self.tamaño)) and (y in range(self.tamaño)):
            if self.estado[x][y]==0:
                return(viable)  

            if self.estado[x][y]==t:
                viable=True
                return (viable)

            x+=h
            y+=k
            

    #intenta poner una ficha del jugador a quien le toque en la posición (i,j)
    def jugada(self,i,j):
        
        self.jugada_viable=False
        self.info=""
        lis=[]

        if self.estavacío(i,j):            
            if self.turno ==Tablero.J2:
                lis=self._buscar_adyacentes(i,j,t=Tablero.J1)
            else:       
                lis=self._buscar_adyacentes(i,j,t=Tablero.J2)
                
        removelist=[]
        for vector in lis:
            if not self._direccion_viable(i,j,vector[0],vector[1]):
                removelist+=[vector]
        for vector in removelist:
            lis.remove(vector)
            
        if not lis:
            self.info="jugada no legal"
            return
        
        self.jugada_viable=True

        #guardar información para tener la opción de deshacer los cambios que se producen en la jugada
        self.turno_saltado=False
        self.estado_anterior=[self.estado[i].copy() for i in range(len(self.estado))]

        for vector in lis:
            self._efecto_dir(i,j,vector[0],vector[1])
        
        self._cambioturno()        
        self._proceder()
    
    
    def _ganador(self):
        nJ1=0
        nJ2=0
        for i in range(self.tamaño):
            for j in range(self.tamaño):
                if self.estado[i][j]==Tablero.J1:
                    nJ1+=1
                if self.estado[i][j]==Tablero.J2:
                    nJ2+=1
        if nJ1==nJ2:
            self.info= self.info+"Empate\n"
        if nJ1<nJ2:
            self.info=self.info+Tablero.J2+ " es el ganador\n"
        if nJ1>nJ2:
            self.info=self.info+Tablero.J1+ " es el ganador\n"
        
        self.info=self.info+str(nJ1)+ "  -  "+ str(nJ2)

    
    def _comprobar_movimientos(self,t=None):

        sin_jugadas=True
        if t==None:
            t=self.turno
        t2=self._turno_opuesto()
        
        #casillas vacías:
        lis0=[]
        for i in range (self.tamaño):
            for j in range(self.tamaño):
                if self.estavacío(i,j):
                    lis0+=[(i,j)]
        
        #adyacentes compatibles
        for jugada in lis0:
            lis1=self._buscar_adyacentes(jugada[0],jugada[1],t2)
            
            for vector in lis1:
                if self._direccion_viable(jugada[0],jugada[1],vector[0],vector[1]):
                    sin_jugadas=False
                    return(sin_jugadas)
                    
        return(sin_jugadas)

        
    # Determina qué jugador debe poner ficha y si acabo el juego.
    def _proceder(self):
        texto=""
        t=self.turno
        t2=self._turno_opuesto()
        
        if (self._comprobar_movimientos(t)):
            texto+=str(self.turno)+' sin movimientos. \n'
            self.info=texto
            self._cambioturno() # GUI se actualiza sola con esto?
            self.turno_saltado=True


            if (self._comprobar_movimientos(t2)):
                texto="Juego finalizado\n"
                self.info=texto
                self._ganador()
                self.fin=True
                
    def deshacer(self):
        self.jugada_viable=True

        if (not(self.estado==self.estado_anterior)) and (self.fin==False):
            self.estado=[self.estado_anterior[i].copy() for i in range(len(self.estado_anterior))]
            self._cambioturno()
            if self.turno_saltado:
                self._cambioturno()
    
    #devuelve la lista de las casillas que cambiaron en el último turno
    def cambios_producidos(self):
        casillas_cambiadas=[]
        for i in range(self.tamaño):
            for j in range(self.tamaño):
                if self.estado_anterior[i][j]!=self.estado[i][j]:
                    casillas_cambiadas+=[(i,j)]
        return(casillas_cambiadas)
                    
                

        

        

In [5]:
root = tk.Tk()

#número de filas/columnas en tablero
SIZE=8

juego=Tablero(SIZE)
root.title("Reversi "+str(SIZE)+"x"+str(SIZE))

#tiempo de espera entre frames de la animación
ESPERA=0.03

#regula el espacio entre casillas
PAD_SIZE=2


#lista de botones e imagenes, que se crearán después. 
botlis=[[None for _ in range (SIZE)] for _ in range(SIZE)]
imaglis=7*[None]

#variables de texto para etiquetas que cambian durante el juego
turno_texto=tk.StringVar()
turno_texto.set("  Turno de "+juego.turno+"  ")
info_texto=tk.StringVar()
info_texto.set("")


#Cargando las imágenes.
for i in range(7):
    imaglis[i]=tk.PhotoImage(file="F"+str(i)+".png")

#diccionario para hacer más ligera la escritura más abajo al representar el tablero
dic_imagenes={Tablero.J1:imaglis[0],Tablero.J2:imaglis[5],0:imaglis[6]}


def table_update():
    
    if (juego.jugada_viable):
        sin_animar=[]
        cambio_añadidos_J1=[]
        cambio_añadidos_J2=[]
        
        for i in range(SIZE):
            for j in range(SIZE):

                if ((juego.estado_anterior[i][j]==juego.estado[i][j]) or
                    (juego.estado[i][j]==0)or(juego.estado_anterior[i][j]==0)):
                    sin_animar+=[(i,j)]
                elif((juego.estado_anterior[i][j]==Tablero.J1) and
                     (Tablero.J2==juego.estado[i][j]) ):
                    cambio_añadidos_J2+=[(i,j)]
                elif((juego.estado_anterior[i][j]==Tablero.J2) and
                     (Tablero.J1==juego.estado[i][j]) ):
                    cambio_añadidos_J1+=[(i,j)]

        for (fil,col) in sin_animar:
            botlis[fil][col].configure(image=dic_imagenes[ juego.estado[fil][col]])

        for fotograma in range(6):
                for (fil,col) in cambio_añadidos_J2:
                    botlis[fil][col].configure(image=imaglis[fotograma])
                for (fil,col) in cambio_añadidos_J1:
                    botlis[fil][col].configure(image=imaglis[5-fotograma])
                time.sleep(ESPERA)
                root.update()

#crea método para intentar colocar una ficha en la casilla i,j
def Jugada(i,j):
    def f():
        if not juego.fin:
            
            global turno_texto
            juego.jugada(i,j)
            table_update()
            turno_texto.set("  Turno de "+juego.turno+"  ")
            info_texto.set(juego.info)

            if (juego.jugada_viable):
                boton_deshacer.configure(state=tk.NORMAL)
            if (juego.fin):
                boton_deshacer.configure(state=tk.DISABLED)
    return(f)


def partida_nueva():
    response=messagebox.askyesno("", "¿Quieres empezar una partida nueva?")
    
    if response:
        info_texto.set("Partida Nueva")
        juego.estado=[[0 for _ in range(SIZE)] for _ in range(SIZE)]
        juego.estado[SIZE//2-1][SIZE//2-1]=Tablero.J2
        juego.estado[SIZE//2][SIZE//2-1]=Tablero.J1
        juego.estado[SIZE//2-1][SIZE//2]=Tablero.J1
        juego.estado[SIZE//2][SIZE//2]=Tablero.J2
        juego.turno=Tablero.J1
        juego.fin=False
        juego.jugada_viable=True
        juego.estado_anterior=[juego.estado[i].copy() for i in range(len(juego.estado))]
        table_update()
        boton_deshacer.configure(state=tk.DISABLED)



#deshacer el último movimiento
def volver_atrás():
    juego.deshacer()
    table_update()
    turno_texto.set("  Turno de "+juego.turno+"  ")
    info_texto.set(juego.info)
    boton_deshacer.configure(state=tk.DISABLED)
    
    
#creando botones
for fil in range(SIZE):
    for col in range(SIZE):
        botlis[fil][col] = tk.Button(root,command= Jugada(fil,col),bg='lightblue',image=imaglis[0])
        botlis[fil][col].grid(row=fil, column=col,pady=PAD_SIZE,padx=PAD_SIZE)

boton_deshacer = tk.Button(root, text="Deshacer    ",state=tk.DISABLED ,command= volver_atrás,bg='lightblue')
boton_deshacer.grid(row=SIZE+2, column=SIZE-1,columnspan=2,pady=PAD_SIZE,padx=PAD_SIZE)
boton_reset = tk.Button(root, text="    Partida nueva" ,command= partida_nueva,bg='lightblue')
boton_reset.grid(row=SIZE+2, column=0,columnspan=2,pady=PAD_SIZE,padx=PAD_SIZE,sticky=tk.W)
        
        
#etiquetas
turno_eti=tk.Label(textvariable =turno_texto,height=2, bg='lightblue',
                   font=("Courier", 14), bd=4, relief=tk.RIDGE)
turno_eti.grid(row=SIZE,columnspan=SIZE,pady=2*PAD_SIZE, sticky=tk.E+tk.W)

info_eti=tk.Label(textvariable =info_texto,height=4,
                   font=("Courier", 12))
info_eti.grid(row=SIZE+1, columnspan=SIZE, pady=2*PAD_SIZE, sticky=tk.E+tk.W)

#cargar imagenes adecuadas
table_update()
root.mainloop()

